LUCAS EE

https://developers.google.com/earth-engine/datasets/catalog/JRC_LUCAS_HARMO_THLOC_V1#citations

TO CHANGE HIERARCHY LEVEL:
- Change sample folder
- Change class values following the right qnt (down in image_classified.set(...))
- Change pallete and classes on Map.addLayer(...) if needed

In [20]:
import ee
import geemap

# Run as service account
service_account = 'ee-account@airy-galaxy-398310.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '../.private-key.json')

ee.Initialize(credentials)

import json
import os
import pandas as pd
import geopandas as gpd
from classifiers import *

#----------------> We initialize some constants for standard use
Map = geemap.Map(center=[43.7196, 10.4250], zoom=5)

countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
Italy = countries.filter(ee.Filter.eq('country_na', 'Italy'))

In [21]:
lucas = ee.FeatureCollection('JRC/LUCAS_HARMO/THLOC/V1') \
  .filterBounds(Italy.geometry()) \
  .select(['point_id', 'lu1', 'lu1_label'])
  
randomLucas = lucas.randomColumn('random')
lucas_fc = randomLucas.sort('random').limit(5000)

# df_filter = df[(df['var'] == 'lu1') | (df['var'] == 'lu1_type')]
# df_code = ee.List(df_filter['code'].tolist())
# Map.addLayer(lucas_fc, {}, 'Points')
# Map

In [22]:
# Reference: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1#bands
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1')

image = ee.Algorithms.Landsat.simpleComposite(**{
    'collection': landsat8\
        .filterBounds(Italy)\
        .filterDate('2018-01-01', '2018-12-31'),
    'asFloat': True
})

In [23]:
folder_path = './level3_samples'

full_df = pd.DataFrame()
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # Read the CSV file into a GeoDataFrame
        filepath = os.path.join(folder_path, filename)
        df = pd.read_csv(filepath)

        full_df = pd.concat([full_df, df], ignore_index=True)

full_df['latitude'] = 0
full_df['longitude'] = 0
del full_df['.geo']

sampledata = geemap.df_to_ee(full_df)
sample = sampledata.randomColumn('random')

print(sample.size().getInfo())

split = 0.8

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

13250


Classes Qnty: 5, 17 and 41

first level = ['', 'U1', 'U2', 'U3', 'U4']

second level = ['', 'U11', 'U12', 'U13', 'U14', 'U15', 'U21', 'U22', 'U31', 'U32', 'U33', 'U34', 'U35', 'U36', 'U37', 'U41', 'U42']

thid level = ['', 'U111', 'U112', 'U113', 'U120', 'U130', 'U140', 'U150', 'U210', 'U221', 'U222', 'U223', 'U224', 'U225', 'U226', 'U227', 'U228', 'U311', 'U312', 'U313', 'U314', 'U315', 'U316', 'U317', 'U318', 'U319', 'U321', 'U322', 'U330', 'U341', 'U342', 'U350', 'U361', 'U362', 'U370', 'U411', 'U412', 'U413', 'U414', 'U415', 'U420']

In [24]:
label = 'landuse'
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

#####################################################
#--> Change Classifier Here: SVM, RandomForest, Cart 
trained = RandomForest.train(training, label, bands)

# Classify image
image_classified = image.classify(trained)

In [25]:
print(image_classified.getInfo())

{'type': 'Image', 'bands': [{'id': 'classification', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -2147483648, 'max': 2147483647}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


Accuracy Types Explanation:

Overall Accuracy essentially tells us out of all of the reference sites what proportion were mapped correctly. The overall accuracy is usually expressed as a percent, with 100% accuracy being a perfect classification where all reference site were classified correctly. Overall accuracy is the easiest to calculate and understand but ultimately only provides the map user and producer with basic accuracy information.

The Kappa Coefficient is generated from a statistical test to evaluate the accuracy of a classification. Kappa essentially evaluates how well the classification performed as compared to just randomly assigning values, i.e. did the classification do better than random. The Kappa Coefficient can range from -1 t0 1. A value of 0 indicated that the classification is no better than a random classification. A negative number indicates the classification is significantly worse than random. A value close to 1 indicates that the classification is significantly better than random.

Producer's Accuracy is the map accuracy from the point of view of the map maker (the producer). This is how often are real features on the ground correctly shown on the classified map or the probability that a certain land cover of an area on the ground is classified as such. The Producer's Accuracy is complement of the Omission Error, Producer's Accuracy = 100%-Omission Error. It is also the number of reference sites classified accurately divided by the total number of reference sites for that class.

The Consumer's Accuracy is the accuracy from the point of view of a map user, not the map maker. the User's accuracy essentially tells use how often the class on the map will actually be present on the ground. This is referred to as reliability. The User's Accuracy is complement of the Commission Error, User's Accuracy = 100%-Commission Error. The User's Accuracy is calculating by taking the total number of correct classifications for a particular class and dividing it by the row total.

In [26]:
train_accuracy = trained.confusionMatrix()

print(train_accuracy.getInfo())
print(train_accuracy.accuracy().getInfo())
print(train_accuracy.kappa().getInfo())
train_produc = [e for l in train_accuracy.producersAccuracy().getInfo() for e in l]
print(train_produc)
print(sum(train_produc)/len(train_produc))
train_consum = [e for l in train_accuracy.consumersAccuracy().getInfo() for e in l]
print(train_consum)
print(sum(train_consum) / len(train_consum))

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 3262, 3, 0, 70, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 15, 0], [0, 102, 93, 1, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0], [0, 15, 3, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0], [0, 106, 4, 0, 4152, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 48, 0], [0, 0, 0, 0, 1, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 4, 0, 0, 4, 0, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 

In [27]:
validated = validation.classify(trained)
test_accuracy = validated.errorMatrix('landuse', 'classification')

print(test_accuracy.getInfo())
print(test_accuracy.accuracy().getInfo())
print(test_accuracy.kappa().getInfo())
test_produc = [e for l in test_accuracy.producersAccuracy().getInfo() for e in l]
print(test_produc)
print(sum(test_produc)/len(test_produc))
test_consum = [e for l in test_accuracy.consumersAccuracy().getInfo() for e in l]
print(test_consum)
print(sum(test_consum) / len(test_consum))

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 804, 8, 3, 85, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 10, 0, 0, 0, 0, 0, 29, 0], [0, 40, 1, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 9, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 113, 3, 0, 905, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 71, 0], [0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 2, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [28]:
image_classified = image_classified.clip(Italy)

Map = geemap.Map(center=[43.710418, 10.396636], zoom=7)
# Map.addLayer(image_classified, {'min': 0, 'max': 40, 'palette': palette}, 'Land Use')
Map.addLayer(image_classified.randomVisualizer(), None, 'Land Use')
Map

Map(center=[43.710418, 10.396636], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [30]:
gpdfile = './ROIs.geojson'
gdf = gpd.read_file(gpdfile)
ee_fc = geemap.gdf_to_ee(gdf)

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_csv = os.path.join(out_dir, 'classification.csv')

# denominator can be used to convert square meters to other areal units, such as square kilimeters
geemap.zonal_statistics_by_group(
    image_classified,
    ee_fc,
    out_csv,
    statistics_type='PERCENTAGE',
    decimal_places=2,
    scale=10
)

SSLError: HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Max retries exceeded with url: /v1/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2384)')))